<a href="https://colab.research.google.com/github/lengochai97/thesis/blob/master/notebooks/feature_construction/21_User_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [0]:
%%capture

import google.colab.drive

google.colab.drive.mount('/content/gdrive', force_remount=True)

# Install Spark and dependencies

In [0]:
import os

os.environ['HADOOP_VERSION'] = '2.7'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/opt/spark'
os.environ['SPARK_VERSION'] = '2.4.3'

In [0]:
%%capture

!wget -qN https://archive.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!tar -xzf spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz -C /opt
!rm spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!rm -rf /opt/spark
!ln -s /opt/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION /opt/spark
!pip install -q findspark

# Create SparkSession

In [0]:
import findspark

findspark.init()

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

# Read files

In [0]:
import json

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
DATA_PATH = '/content/gdrive/My Drive/dataset/adressa/one_week'

In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'clean.json')) as file:
  clean_schema = T.StructType.fromJson(json.load(file))

In [0]:
df_clean = spark.read.json(os.path.join(DATA_PATH, 'clean'), schema=clean_schema)

In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'news_features.json')) as file:
  news_features_schema = T.StructType.fromJson(json.load(file))

In [0]:
df_news_features = spark.read.json(os.path.join(DATA_PATH, 'news_features'), schema=news_features_schema)

# Construct user features

In [0]:
df_event = (
    df_clean
    .select(
        F.column('userId'),
        F.column('time'),
        F.column('newsId'),
    )
    .join(
        df_news_features
        .select(
            F.column('newsId'),
            F.column('categoryList'),
        ),
        on='newsId',
        how='inner',
    )
)

In [0]:
from pyspark.sql import Window

In [0]:
def build_user_profile(df_event, time_tag, time_delta):
  df_user_profile = (
      df_event
      .withColumn(
          'categoryListList',
          F.collect_list(F.column('categoryList')).over(
              Window
              .partitionBy('userId')
              .orderBy('time')
              .rangeBetween(-time_delta, -1)
          ),
      )
      .withColumn(
          f'userClickCount{time_tag}',
          F.size(F.column('categoryListList')),
      )
      .withColumn(
          f'categoryList{time_tag}',
          F.array_distinct(F.flatten(F.column('categoryListList'))),
      )
      .select(
          F.column('userId'),
          F.column('time'),
          F.column(f'userClickCount{time_tag}'),
          F.column(f'categoryList{time_tag}'),
      )
  )
  
  return df_user_profile

In [0]:
def build_user_history(df_event):
  df_user_history = (
      df_event
      .withColumn(
          'categoryListList',
          F.collect_list(F.column('categoryList')).over(
              Window
              .partitionBy('userId')
              .orderBy('time')
              .rangeBetween(Window.unboundedPreceding, -1)
          ),
      )
      .withColumn(
          'categoryListHistory',
          F.flatten(F.column('categoryListList')),
      )
      .select(
          F.column('userId'),
          F.column('time'),
          F.column('categoryListHistory'),
      )
  )
  
  return df_user_history

In [0]:
time_info = (
    ('1H', 3600),
    ('6H', 21600),
    ('1D', 86400),
    ('1W', 604800),
)

In [0]:
df_user_features = (
    df_event
    .select(
        F.column('userId'),
        F.column('time'),
    )
)

for time_tag, time_delta in time_info:
  df_user_features = (
      df_user_features
      .join(
          build_user_profile(df_event, time_tag, time_delta),
          on=['userId', 'time'],
          how='inner',
      )
  )

df_user_features = (
    df_user_features
    .join(
        build_user_history(df_event),
        on=['userId', 'time'],
        how='inner',
    )
)

# Write files

In [0]:
df_user_features = (
    df_user_features
    .repartition(F.column('userId'))
    .sortWithinPartitions(
        F.column('time'),
        F.column('userId'),
    )
)

In [0]:
%%time

df_user_features.write.json(os.path.join(DATA_PATH, 'user_features'))

CPU times: user 23.2 ms, sys: 5.46 ms, total: 28.6 ms
Wall time: 2min 11s


In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'user_features.json'), 'w+') as file:
  json.dump(df_user_features.schema.jsonValue(), file)